In [1]:
"""
Code to produce the results in section 4.5 of the paper 
Boltzmann Convolutions and Welford mean-variance layers with an application to 
time series forecasting and classification
"""

'\nCode to produce the results in section 4.5 of the paper \nBoltzmann Convolutions and Welford mean-variance layers with an application to \ntime series forecasting and classification\n'

In [2]:
#Import packages to facillitate this research 
import numpy as np 
import tensorflow as tf #(https://www.tensorflow.org/install)
from tensorflow.keras import (layers,Model) 
from tensorflow.keras.layers import (Layer, Input, Dense, Concatenate, Activation, TimeDistributed,
Flatten, Lambda, GlobalAveragePooling1D, LeakyReLU) 
from tensorflow.keras.utils import to_categorical 
from concretedropout.tensorflow import (ConcreteDenseDropout, ConcreteSpatialDropout1D, # https://github.com/aurelio-amerio/ConcreteDropout
get_weight_regularizer, get_dropout_regularizer) 
from sktime.classification.deep_learning import InceptionTimeClassifier #https://www.sktime.net/en/stable/get_started.html
from aeon.classification.convolution_based import MultiRocketHydraClassifier #https://www.aeon-toolkit.org/en/stable/installation.html
from aeon.classification.early_classification import TEASER
from aeon.classification.hybrid import HIVECOTEV2
from sklearn.metrics import (accuracy_score, mean_absolute_error, mean_squared_error,  # https://scikit-learn.org/stable/install.html
classification_report)
from sklearn.model_selection import train_test_split
from math import comb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.12/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [3]:
def generate_ar3(n, coeffs, noise_std=1.0):
    """
    Function to generate a single realization consisting of n observations from a autoregressive process of order 3.
    Coefficients given in coefs, where the index of a coefficient determines its associated lag 
    e.g. index 0 coefficient is the coefficient associated with the first lag. 
    noise_std determines the variance of the white noise in the processes. 
    """
    phi1, phi2, phi3 = coeffs
    noise = np.random.normal(scale=noise_std, size=n)
    x = np.zeros(n)
    for t in range(3, n):
        x[t] = phi1 * x[t-1] + phi2 * x[t-2] + phi3 * x[t-3] + noise[t]
    return x
def generate_ar2(n, coeffs, noise_std=1.0):
    """
    Function to generate a single realization consisting of n observations from a autoregressive process of order 2.
    Coefficients given in coefs, where the index of a coefficient determines its associated lag 
    e.g. index 0 coefficient is the coefficient associated with the first lag. 
    noise_std determines the variance of the white noise in the processes. 
    """
    phi1, phi2 = coeffs
    noise = np.random.normal(scale=noise_std, size=n)
    x = np.zeros(n)
    for t in range(2, n):
        x[t] = phi1 * x[t-1] + phi2 * x[t-2] + noise[t]
    return x
n_series = 10000  
n_time_points = 50  
ar3_coeffs = [0.6, -0.3, 0.2]
ar2_coeffs = [0.7, -0.4]
labels = np.random.choice([0, 1], size=n_series)
time_series = []
for label in labels:
    if label == 1:  # AR(3)
        time_series.append(generate_ar3(n_time_points, ar3_coeffs))
    else:  # AR(2)
        time_series.append(generate_ar2(n_time_points, ar2_coeffs))
time_series = np.array(time_series)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(time_series, labels, test_size=0.2, random_state=42) #train-test split 
X_train = np.expand_dims(X_train, axis = -1) 
X_test = np.expand_dims(X_test, axis = -1) 
y_train = to_categorical(y_train, num_classes=2) #training one-hot encoding of class labels 
y_test= to_categorical(y_test, num_classes=2) #testing one-hot encoding of class labels 
n_observed = 40 #number of time points which are observed with subsequent time points being forecasted by ForeClassNet 
X_observed_train = X_train[:, :n_observed, :] #training input into the models
X_forecast_train = X_train[:, n_observed:, :] #training forecasting output of ForeClassNet
X_observed_test = X_test[:, :n_observed, :] #test input into the models
X_forecast_test = X_test[:, n_observed:, :] #test forecasting output of ForeClassNet

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_observed_train.shape)
print(X_observed_test.shape)
print(X_forecast_train.shape)
print(X_forecast_test.shape)

(8000, 50, 1)
(2000, 50, 1)
(8000, 2)
(2000, 2)
(8000, 40, 1)
(2000, 40, 1)
(8000, 10, 1)
(2000, 10, 1)


In [6]:
Ns = X_observed_train.shape[0]
time_steps = X_observed_train.shape[1]
n_forecast = X_forecast_train.shape[1]
n_classes = y_train.shape[1]
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet

In [7]:
def create_custom_filters(filter_length):
    """
    Function that takes in a user specified filter length to then construct hand-crafted filters 
    according to (8)-(10) in section 3.2 of the research paper. 
    """
    filter_1 = np.array([(-1) ** (i + 1) for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Decreasing filter
    filter_2 = np.array([(-1) ** i for i in range(filter_length)], dtype=np.float32).reshape(filter_length, 1, 1) #Increasing filter
    filter_3 = np.zeros(filter_length) #Peaks filter 
    for n in range(0, int(((filter_length - 3) / 3) + 1)): #see section 3.2 for discussion of the formulas
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), n)
    for n in range(int(filter_length / 3), int((2 * (filter_length - 3) / 3) + 2)):
        filter_3[n] = (6 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (filter_length / 3)))
    for n in range(int((2 * (filter_length - 3) / 3) + 2), filter_length):
        filter_3[n] = (-3 / filter_length) * comb(int((filter_length - 3) / 3), int(n - (2 * (filter_length - 3) / 3) - 2))
    filter_3 = filter_3.reshape(filter_length, 1, 1)
    custom_filters = np.concatenate([filter_1, filter_2, filter_3], axis=2) #combine the filters together
    return custom_filters
class WelfordMeanVarianceLayer(Layer): #Defining the Welford Mean-variance layer (section 2.2) as a subclass of Layer in Keras 
    def __init__(self, total_series, n_forecast=4, **kwargs): #constructor of the class, where it takes in the number of time series we want to have mean and variance estimates for and the number of forecasts for each 
        super(WelfordMeanVarianceLayer, self).__init__(**kwargs) #calls the Keras layer class
        self.total_series = total_series #number of series the layer will track 
        self.n_forecast = n_forecast #number of forecast steps ahead 
        self.stats = self.add_weight( #creates a non-trainable tensor to store the means and variances which is initialized with zeros
            shape=(self.total_series, self.n_forecast, 2),
            initializer="zeros",
            trainable=False,
            name="stats"
        ) 
        self.count = self.add_weight( #creates a non-trainable tensor to count how many samples for the mean and variance we have for the forecasts of a given time series, which are obtained through successive forward passes
            shape=(self.total_series,),
            initializer="zeros",
            trainable=False,
            name="count"
        )
    def reset_statistics(self): #A method to clear the existing estimates of the mean and variance of the forecasts and set them to zero, which you might want to do during testing
        self.stats.assign(tf.zeros_like(self.stats))
        self.count.assign(tf.zeros_like(self.count))
    def call(self, inputs, **kwargs): #main method used when calling the Welford layer in a given forward pass
        forecast_values, series_ids = inputs  #unpacks the inputs into the Welford layer, the forecast values in the current forward pass and the ids of the current time series being processed 
        batch_size = tf.shape(forecast_values)[0] #derives the current batch size from the shape of the forecast values tensor
        def loop_body(i, current_stats, current_count): #defines the body of a while loop to be used in the proceeding lines. Processes the ith time series in the batch by applying Welford's algorithm 
            idx = series_ids[i] #determines the id of the current time series being processed        
            val = forecast_values[i] #the values to be used to update the means and variances of the forecasts
            current_count = tf.tensor_scatter_nd_add(current_count, [[idx]], [1.0]) #increments the count of the number of times we have updated the mean and variance of the given time series 
            old_mean = current_stats[idx, :, 0] #extracts the old forecast means 
            old_var  = current_stats[idx, :, 1] #extracts the old forecast variance 
            n = current_count[idx] #number of times we have updated the mean and variance of the forecasts for a given time series 
            delta = val - old_mean #the difference between the new forecast value and the old mean 
            new_mean = old_mean + (delta /n) #formula used in Welford's algorithm to update the mean 
            delta2 = val - new_mean #difference between the new forecast value and the new mean 
            new_var = ((1-(1/n))*old_var) + ((delta*delta2)/n) #update the value of the sample variance using the population variance formula      
            updated_stats = tf.stack([new_mean, new_var], axis=-1) #combines the new values of the sample mean and variance 
            current_stats = tf.tensor_scatter_nd_update(current_stats, [[idx]], [updated_stats]) #overwrites the old values of the mean and variance for a given time series(idx) with the new values of the mean and variance 
            return i+1, current_stats, current_count #set i to i+1 to go to the next time series in the batch, and return the current values of the sample mean and variance, and the number of times the time series have been processed
        i0 = tf.constant(0) #starting index of the loop 
        stats0 = self.stats #the initial stats for the start of the while loop 
        count0 = self.count #the initial counts that each time series has been processed for the start of the while loop 
        _, updated_stats, updated_count = tf.while_loop( #the while loop which outputs the index, the updated stats, and the updated counts 
            cond=lambda i, *_: i < batch_size, #determines if the while loop should continue or not, where the loop stops if i exceeds the batch size as there are no more time series to process in the current batch 
            body=loop_body, #using the function defined earlier which is used in each iteration of the while loop 
            loop_vars=[i0, stats0, count0] #initial values of the variables in the while loop 
        )
        self.stats.assign(updated_stats) #permanently updates the stats variable of the layer which will be used in future calls of the layer 
        self.count.assign(updated_count) #permanently updates the count variable for the layer which will be used in future calls of the layer 
        out_stats_for_batch = tf.gather(self.stats, series_ids) #takes the means and variances for the forecasted time points for the time series in the current batch so they can be passed to the proceeding layer 
        return out_stats_for_batch #returns the means and variances for the forecasted time series in the current batch so they can be passed into the next layer of the network in the current forward pass 
class BoltzmannConv1D(Layer): #Defining the Boltzmann convolutional layer (section 2.1) as a subclass of Layer in Keras 
    def __init__(self, out_channels, kernel_sizes,wr,dr,dilation_rate=1, padding="causal", temperature=1, **kwargs):#contructor of the class which takes in several hyperparameters including the type of padding, dilation rate, and temperature of the Boltzmann distribution
        super(BoltzmannConv1D, self).__init__(**kwargs) #Calls the Keras layer class 
        self.out_channels = out_channels #number of output channels 
        self.kernel_sizes = kernel_sizes #the set of kernel filter lengths 
        self.wr = wr
        self.dr = dr
        self.dilation_rate = dilation_rate #dilation rate of the layer 
        self.padding = padding #type of padding for the layer 
        self.temperature = temperature #value of the temperature for the Boltzmann distribution 
        self.convs = [  #creates a list consisting of sub-convolutional layers with the parameters defined above. The layer is wrapped in a Spatial Concrete Dropout layer 
            ConcreteSpatialDropout1D(
                tf.keras.layers.Conv1D(
                    out_channels, ks,
                    dilation_rate=dilation_rate,
                    padding=padding
                ),
                is_mc_dropout=True,
                weight_regularizer=self.wr,
                dropout_regularizer=self.dr
            )
            for ks in kernel_sizes #iterates over the filter lengths in the kernel_sizes list creating associated convolutional layers
        ]
    def build(self, input_shape): #create the weights used in the layers 
        self.probs = self.add_weight(   #logits used in the Boltzmann convolution distribution  which are learnable 
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(BoltzmannConv1D, self).build(input_shape) #finalizes the build method by inheriting the build from the overall Keras layer class 
    def call(self, x, training=False): #main method used when calling the Boltzmann layer for a given forward pass 
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature) #defines the probability discussed in section 2.1 
        conv_outputs = [ #creates the list of the weighted convolution outputs 
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities)) #turns the kernel probabilities into a list which are then paired with their associated convolutional layer, with them being multiplied together 
        ]
        out = tf.add_n(conv_outputs) #sums up the weighted convolutional outputs in the list 
        return out #returns the summed output of the weighted convolutional layers according to (2) in section 2.1 
class PredefinedBoltzmannConv1D(Layer):
    def __init__(self, filters, kernel_sizes, dilation_rate, custom_filters, temperature=1, **kwargs):
        super(PredefinedBoltzmannConv1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_sizes = kernel_sizes
        self.temperature = temperature
        self.custom_filters = custom_filters #pre-defined filters as described in (8)-(10) in section 3.2
        self.dilation_rate = dilation_rate
        self.convs = []
        for ks in kernel_sizes:
            conv_layer = tf.keras.layers.Conv1D(
                filters=filters,
                kernel_size=ks,
                padding="causal",
                use_bias=False,
                dilation_rate=dilation_rate
            )
            self.convs.append(conv_layer)
    def build(self, input_shape):
        for i, (conv_layer, ks) in enumerate(zip(self.convs, self.kernel_sizes)):
            conv_layer.build(input_shape)
            filter_weights = self.custom_filters[ks]
            conv_layer.set_weights([filter_weights])
            conv_layer.trainable = False
        self.probs = self.add_weight(
            shape=(len(self.kernel_sizes),),
            initializer="uniform",
            trainable=True,
            name="kernel_probs"
        )
        super(PredefinedBoltzmannConv1D, self).build(input_shape)
    def call(self, x, training=False):
        kernel_probabilities = tf.nn.softmax(self.probs / self.temperature)
        conv_outputs = [
            conv(x) * kernel_prob
            for conv, kernel_prob in zip(self.convs, tf.unstack(kernel_probabilities))
        ]
        out = tf.add_n(conv_outputs)
        return out

In [8]:
kernel_sizes = [3,6,12,18,24,30,36] #pre-defined filter lengths (section 2.1)
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes} #pre-defined filter weights as a function of filter length (8)-(10) in section 3.2
ts_input = Input(shape=(time_steps, 1), name="ts_input")   #defining the input time series to ForeClassNet       
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")   #defining the input time series index to ForeClassNet 

#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1, wr = wr, dr = dr)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2, wr = wr, dr = dr)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4, wr = wr, dr = dr)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connectedlayer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 10000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8, wr = wr, dr = dr)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 40, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 40, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 40, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 40, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 40, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 40, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 40, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 40, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 40)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,313 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 40, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 10)        │        331 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 40, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,584,480 (6.04 MB)

 Trainable params: 1,374,093 (5.24 MB)

 Non-trainable params: 210,387 (821.82 KB)

In [9]:
#Train ForeClassNet
batch_size = 64
epochs = 100
tic = time.perf_counter()
series_id_train = np.arange(Ns, dtype=np.int32)
model.fit(x=[X_observed_train, series_id_train],  y=[X_forecast_train, y_train], batch_size=batch_size,
    epochs=epochs,validation_split=0.1,shuffle=True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 27s 146ms/step - activation_accuracy: 0.5022 - activation_loss: 0.6931 - loss: 1.9709 - output_forecast_loss: 1.5067 - output_forecast_mse: 1.5067 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6922 - val_loss: 1.9267 - val_output_forecast_loss: 1.4651 - val_output_forecast_mse: 1.4752
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 15s 135ms/step - activation_accuracy: 0.5150 - activation_loss: 0.6929 - loss: 1.9209 - output_forecast_loss: 1.4727 - output_forecast_mse: 1.4727 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6925 - val_loss: 1.9106 - val_output_forecast_loss: 1.4651 - val_output_forecast_mse: 1.4752
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 16s 140ms/step - activation_accuracy: 0.5169 - activation_loss: 0.6928 - loss: 1.9244 - output_forecast_loss: 1.4929 - output_forecast_mse: 1.4929 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6921 - val_loss: 1.8928 - val_output_forecast_loss: 1.4649 - val_output_f

In [10]:
def mc_dropout_predict(model, X_test, test_ids, num_samples=100):
    """
    Peforms Monte Carlo dropout, where dropout is applied at inference time. 
    This takes as input the trained ForeClassNet model, the input observed time series, 
    the id's associated with the time series in the test set, and the number of MC samples which
    is the number of forward passes through the trained ForeClassNet model. 
    """
    forecast_predictions = [] #list to store the forecast output from each forward pass through ForeClassNet
    classification_predictions = [] #list to store the classification output from each forward pass through ForeClassNet
    for _ in tqdm(range(num_samples), desc="MC Dropout passes"):
        forecast, classification = model.predict([X_test, test_ids], verbose=0) #performs a forward pass through the network 
        forecast_predictions.append(forecast) #add the forecast array to the list 
        classification_predictions.append(classification) #add the classification array of probabilities to the list
    forecast_predictions = np.array(forecast_predictions)
    classification_predictions = np.array(classification_predictions) 
    mean_forecast = np.mean(forecast_predictions, axis=0) #take the mean with respect to the MC samples of the forecast output
    std_forecast = np.std(forecast_predictions, axis=0) #take the standard deviation with respect to the MC samples of the forecast output
    mean_classification = np.mean(classification_predictions, axis=0) #take the mean with respect to the MC samples of the classification probabilities
    std_classification = np.std(classification_predictions, axis=0) #take the standard deviation with respect to the MC samples of the classification probabilities
    return (mean_forecast, std_forecast), (mean_classification, std_classification)

In [11]:
def generate_adversarial_examples(model, x,series_ids, y, epsilon=0.1):
    """
    Function used to generate adversarial examples using the FGSM with respect to the 
    categorical cross entropy loss function. 
    Takes as input the trained ForeClasNet model, the training examples,x, you want to manipulate and
    their associated labels y, and epsilon determines the magnitude of the attack. 
    """
    x = tf.convert_to_tensor(x, dtype=tf.float32) #convert to tensor to ensure tensor flow can track gradient computations
    series_ids = tf.convert_to_tensor(series_ids, dtype=tf.int32)
    with tf.GradientTape() as tape: #create context to track computations
        tape.watch(x) #ensure x is tracked 
        predictions = model([x,series_ids], training = False) #perform a forward pass on the trained model with a given input example x
        classification_output = predictions[1] #extract the classification output from the forward pass
        loss = tf.keras.losses.categorical_crossentropy(y, classification_output) #compute the loss for the given example

    gradients = tape.gradient(loss, x) #compute gradients
    perturbation = epsilon * tf.sign(gradients) #compute adversarial perturbation 
    adversarial_x = tf.clip_by_value(x + perturbation, tf.reduce_min(x), tf.reduce_max(x)) #create adversarial example within the realms of the data 
    return adversarial_x.numpy()

In [12]:
#generate adversarial test set data for the first trained ForeClassNet 
epsilon = 0.1  
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
adversarial_test_set = generate_adversarial_examples(model, X_observed_test,test_series_ids,y_test )
#assess the performance of the first trained ForeClassNet on this adversarial test set 
n_test_series = adversarial_test_set.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer
# Perform MC Dropout predictions with your test data and test IDs
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [03:11<00:00,  1.91s/it]

Predicted classes: [0 0 0 ... 0 0 0]
0.489
              precision    recall  f1-score   support

           0       0.49      0.99      0.66       991
           1       0.00      0.00      0.00      1009

    accuracy                           0.49      2000
   macro avg       0.25      0.49      0.33      2000
weighted avg       0.24      0.49      0.33      2000



In [13]:
clf = InceptionTimeClassifier(n_epochs=100, batch_size = 64) #Initialize the inception time classifier 
tic = time.perf_counter() #time how long it takes for training 
clf.fit(X_observed_train.transpose(0,2,1), np.argmax(y_train,axis = 1)) #train the inception time classifier 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") #how long the inception time classifier took to train 
predictions = clf.predict(adversarial_test_set.transpose(0,2,1)) #evaluating the trained inception time classifier on the observed test set (excludes forecast time points)
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions) #evaluate its performance on the test set 
print(classification__report)

Time taken for fitting: 780.79 seconds
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Classification Test Accuracy: 0.4845
              precision    recall  f1-score   support

           0       0.49      0.95      0.65       991
           1       0.36      0.03      0.05      1009

    accuracy                           0.48      2000
   macro avg       0.43      0.49      0.35      2000
weighted avg       0.43      0.48      0.35      2000



In [14]:
MRhydra = MultiRocketHydraClassifier() #initialize the MR-Hydra classifier 
MRhydra.fit(X_observed_train.transpose(0, 2, 1), np.argmax(y_train,axis = 1))  #train the MR-Hydra classifier on the observed time series 
predictions = MRhydra.predict(adversarial_test_set.transpose(0, 2, 1)) #test the trained MR-Hydra classifier on the observed test set 
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

0.7655
              precision    recall  f1-score   support

           0       0.77      0.75      0.76       991
           1       0.76      0.78      0.77      1009

    accuracy                           0.77      2000
   macro avg       0.77      0.77      0.77      2000
weighted avg       0.77      0.77      0.77      2000



In [15]:
early = TEASER(classification_points=[40]) #initalize the teaser classifier, specifying we want to make the classification decision after we have observed 40 time points 
tic = time.perf_counter()
early.fit(X_train.transpose(0,2,1), np.argmax(y_train,axis = 1)) #train the teaser classifier 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
predictions, decisions = early.predict(adversarial_test_set.transpose(0,2,1)) #evaluate the trained teaser classifier 
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 258.95 seconds
Classification Test Accuracy: 0.6975
              precision    recall  f1-score   support

           0       0.69      0.70      0.70       991
           1       0.70      0.69      0.70      1009

    accuracy                           0.70      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.70      0.70      0.70      2000



In [16]:
hc2 = HIVECOTEV2(time_limit_in_minutes= time_in_mins)
y_train_labels = np.argmax(y_train, axis=1)
tic = time.perf_counter()
hc2.fit(X_observed_train.transpose(0, 2, 1),y_train_labels)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
predictions = hc2.predict(adversarial_test_set.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 2892.37 seconds
Classification Test Accuracy: 0.7475
              precision    recall  f1-score   support

           0       0.72      0.81      0.76       991
           1       0.78      0.69      0.73      1009

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



In [17]:
#generate adversarial training data
epsilon = 0.1  
n_test_series = X_observed_train.shape[0]
train_series_ids = np.arange(n_test_series, dtype=np.int32)
adversarial_train_set = generate_adversarial_examples(model, X_observed_train, train_series_ids,y_train)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer
model = 0 
tf.keras.backend.clear_session()
kernel_sizes = [3,6,12,18,24,30,36] 
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes}
ts_input = Input(shape=(time_steps, 1), name="ts_input")         
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")    
Ns = adversarial_train_set.shape[0]
time_steps = adversarial_train_set.shape[1]
n_forecast = X_forecast_train.shape[1]
n_classes = y_train.shape[1]
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) #Determines the amount of regularization on the weights in the concrete dropout layers of ForeClassNet
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) #Determines the amount of regularization on the dropout probabilities of ForeClassNet
#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1, wr = wr, dr = dr)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2, wr = wr, dr = dr)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4, wr = wr, dr = dr)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connectedlayer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 14000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8, wr = wr, dr = dr)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 40, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 40, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 40, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 40, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 40, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 40, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 40, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 40, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 40)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,313 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 40, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 10)        │        331 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 40, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,668,480 (6.36 MB)

 Trainable params: 1,374,093 (5.24 MB)

 Non-trainable params: 294,387 (1.12 MB)

In [18]:
print(adversarial_train_set.shape)
print(X_observed_test.shape)
print(adversarial_test_set.shape)

(8000, 40, 1)
(2000, 40, 1)
(2000, 40, 1)


In [19]:
#Train ForeClassNet
batch_size = 64
epochs = 100
tic = time.perf_counter()
series_id_train = np.arange(Ns, dtype=np.int32)
model.fit(x=[adversarial_train_set, series_id_train],  y=[X_forecast_train, y_train], batch_size=batch_size,
    epochs=epochs,validation_split=0.1,shuffle=True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 26s 135ms/step - activation_accuracy: 0.5032 - activation_loss: 0.6932 - loss: 1.9794 - output_forecast_loss: 1.5151 - output_forecast_mse: 1.5151 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6925 - val_loss: 1.9263 - val_output_forecast_loss: 1.4645 - val_output_forecast_mse: 1.4745
Epoch 2/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 15s 137ms/step - activation_accuracy: 0.5016 - activation_loss: 0.6931 - loss: 1.9486 - output_forecast_loss: 1.5003 - output_forecast_mse: 1.5003 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6923 - val_loss: 1.9099 - val_output_forecast_loss: 1.4649 - val_output_forecast_mse: 1.4749
Epoch 3/100
113/113 ━━━━━━━━━━━━━━━━━━━━ 16s 138ms/step - activation_accuracy: 0.5095 - activation_loss: 0.6929 - loss: 1.9359 - output_forecast_loss: 1.5045 - output_forecast_mse: 1.5045 - val_activation_accuracy: 0.5350 - val_activation_loss: 0.6922 - val_loss: 1.8925 - val_output_forecast_loss: 1.4649 - val_output_f

In [20]:
epsilon = 0.1  
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
adversarial_test_set_new = generate_adversarial_examples(model, X_observed_test,test_series_ids,y_test )

In [21]:
#(b)
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,X_observed_test,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)
#(a)
n_test_series = adversarial_test_set.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)
#(c)
n_test_series = adversarial_test_set_new.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set_new,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [02:50<00:00,  1.70s/it]


Predicted classes: [0 0 0 ... 0 0 0]
0.5165
              precision    recall  f1-score   support

           0       0.51      0.98      0.67       991
           1       0.74      0.06      0.12      1009

    accuracy                           0.52      2000
   macro avg       0.63      0.52      0.39      2000
weighted avg       0.63      0.52      0.39      2000



MC Dropout passes: 100%|██████████████████████| 100/100 [02:53<00:00,  1.73s/it]


Predicted classes: [0 1 0 ... 1 0 1]
0.9965
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       991
           1       1.00      1.00      1.00      1009

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



MC Dropout passes: 100%|██████████████████████| 100/100 [02:51<00:00,  1.72s/it]

Predicted classes: [0 0 1 ... 0 0 0]
0.438
              precision    recall  f1-score   support

           0       0.46      0.83      0.59       991
           1       0.24      0.05      0.09      1009

    accuracy                           0.44      2000
   macro avg       0.35      0.44      0.34      2000
weighted avg       0.35      0.44      0.34      2000



In [22]:
clf = 0 
clf = InceptionTimeClassifier(n_epochs=100, batch_size = 64)  
tic = time.perf_counter() 
clf.fit(adversarial_train_set.transpose(0,2,1), np.argmax(y_train,axis = 1)) 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") 
#(b)
predictions = clf.predict(X_observed_test.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)
#(a)
predictions = clf.predict(adversarial_test_set.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)
#(c)
predictions = clf.predict(adversarial_test_set_new.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)

Time taken for fitting: 731.42 seconds
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Classification Test Accuracy: 0.496
              precision    recall  f1-score   support

           0       0.50      0.98      0.66       991
           1       0.51      0.02      0.04      1009

    accuracy                           0.50      2000
   macro avg       0.50      0.50      0.35      2000
weighted avg       0.50      0.50      0.35      2000

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Classification Test Accuracy: 0.987
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       991
           1       0.99      0.99      0.99      1009

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
Classification Test Accuracy: 0.45
              precision    recall  f1-score   support

           0       0.47  

In [23]:
MRhydra = 0 
MRhydra = MultiRocketHydraClassifier()  
MRhydra.fit(adversarial_train_set.transpose(0, 2, 1), np.argmax(y_train,axis = 1))  
#(b)
predictions = MRhydra.predict(X_observed_test.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions = MRhydra.predict(adversarial_test_set.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions = MRhydra.predict(adversarial_test_set_new.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

0.7615
              precision    recall  f1-score   support

           0       0.73      0.83      0.78       991
           1       0.81      0.69      0.75      1009

    accuracy                           0.76      2000
   macro avg       0.77      0.76      0.76      2000
weighted avg       0.77      0.76      0.76      2000

0.8065
              precision    recall  f1-score   support

           0       0.80      0.81      0.81       991
           1       0.81      0.80      0.81      1009

    accuracy                           0.81      2000
   macro avg       0.81      0.81      0.81      2000
weighted avg       0.81      0.81      0.81      2000

0.674
              precision    recall  f1-score   support

           0       0.65      0.75      0.69       991
           1       0.71      0.60      0.65      1009

    accuracy                           0.67      2000
   macro avg       0.68      0.67      0.67      2000
weighted avg       0.68      0.67      0.67      2000


In [24]:
early = 0 
early = TEASER(classification_points=[40])
tic = time.perf_counter()
early.fit(adversarial_train_set.transpose(0,2,1), np.argmax(y_train,axis = 1)) 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
#(b)
predictions, decisions = early.predict(X_observed_test.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions, decisions = early.predict(adversarial_test_set.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions, decisions = early.predict(adversarial_test_set_new.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 118.16 seconds
Classification Test Accuracy: 0.7145
              precision    recall  f1-score   support

           0       0.70      0.75      0.72       991
           1       0.73      0.68      0.71      1009

    accuracy                           0.71      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.72      0.71      0.71      2000

Classification Test Accuracy: 0.697
              precision    recall  f1-score   support

           0       0.68      0.72      0.70       991
           1       0.71      0.67      0.69      1009

    accuracy                           0.70      2000
   macro avg       0.70      0.70      0.70      2000
weighted avg       0.70      0.70      0.70      2000

Classification Test Accuracy: 0.691
              precision    recall  f1-score   support

           0       0.68      0.72      0.70       991
           1       0.71      0.67      0.69      1009

    accuracy                      

In [25]:
hc2 = 0 
hc2 = HIVECOTEV2(time_limit_in_minutes= time_in_mins)
y_train_labels = np.argmax(y_train, axis=1)
tic = time.perf_counter()
hc2.fit(adversarial_train_set.transpose(0, 2, 1),y_train_labels)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
#(b)
predictions = hc2.predict(X_observed_test.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions = hc2.predict(adversarial_test_set.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions = hc2.predict(adversarial_test_set_new.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 2820.88 seconds
Classification Test Accuracy: 0.8345
              precision    recall  f1-score   support

           0       0.80      0.89      0.84       991
           1       0.88      0.78      0.83      1009

    accuracy                           0.83      2000
   macro avg       0.84      0.83      0.83      2000
weighted avg       0.84      0.83      0.83      2000

Classification Test Accuracy: 0.8535
              precision    recall  f1-score   support

           0       0.85      0.86      0.85       991
           1       0.86      0.85      0.85      1009

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000

Classification Test Accuracy: 0.792
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       991
           1       0.83      0.74      0.78      1009

    accuracy                    

In [26]:
combined_X_train = np.concatenate([X_observed_train, adversarial_train_set], axis=0)
combined_X_forecast_train = np.concatenate([X_forecast_train,X_forecast_train],axis = 0 )
combined_y_train = np.concatenate([y_train, y_train], axis=0)
indices = np.arange(combined_X_train.shape[0])
np.random.shuffle(indices)
combined_X_train = combined_X_train[indices]
combined_y_train = combined_y_train[indices]
combined_X_forecast_train = combined_X_forecast_train[indices]
print(combined_X_train.shape)
print(X_observed_test.shape)
print(adversarial_test_set.shape)
print(adversarial_test_set_new.shape)


(16000, 40, 1)
(2000, 40, 1)
(2000, 40, 1)
(2000, 40, 1)


In [27]:
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics() #reset the statistics in the Welford layer
model = 0 
tf.keras.backend.clear_session()
kernel_sizes = [3,6,12,18,24,30,36] 
custom_filters_dict = {ks: create_custom_filters(ks) for ks in kernel_sizes}
ts_input = Input(shape=(time_steps, 1), name="ts_input")         
series_id_input = Input(shape=(), dtype=tf.int32, name="series_id")    
Ns = combined_X_train.shape[0]
time_steps = combined_X_train.shape[1]
n_forecast = combined_X_forecast_train.shape[1]
n_classes = combined_y_train.shape[1]
wr = get_weight_regularizer(Ns, l=1e-2, tau=0.5) 
dr = get_dropout_regularizer(Ns, tau=0.5, cross_entropy_loss=True) 
#Parallel layer and concatenation layer#
conv_predefined = PredefinedBoltzmannConv1D(filters=3, kernel_sizes=kernel_sizes,dilation_rate=1, custom_filters=custom_filters_dict)
x1 = conv_predefined(ts_input)
x1 = Activation("relu", name="activation_predefined")(x1)
x2 = BoltzmannConv1D(out_channels=128, kernel_sizes=kernel_sizes, dilation_rate=1, wr = wr, dr = dr)(ts_input)
x2 = Activation("relu", name="activation_learnable")(x2)
concatenated = Concatenate(axis=-1)([x1, x2])

#Boltzmann Convolutional layer#
x3 = BoltzmannConv1D(out_channels=64, kernel_sizes=kernel_sizes, dilation_rate=2, wr = wr, dr = dr)(concatenated)
x3 = Activation("relu", name="activation_2")(x3)

#Boltzmann convolutional layer#
x4 = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes, dilation_rate=4, wr = wr, dr = dr)(x3)
x4 = Activation("relu", name="activation_3")(x4)

#Time distributed dense layer#
layer_4 = Dense(1, use_bias=False)
layer_4 = ConcreteDenseDropout(layer_4, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr)
x5 = TimeDistributed(layer_4)(x4)  
x5 = Flatten()(x5)                 
x5_expanded = Lambda(lambda x: tf.expand_dims(x, axis=-1))(x5)
zeros_tensor = Lambda(lambda x: tf.zeros_like(x))(x5_expanded)
x6_final = Concatenate(axis=-1)([x5_expanded, zeros_tensor])

#Fully connectedlayer#
layer_6 = Dense(32, use_bias=True)  
layer_6 = ConcreteDenseDropout(layer_6, is_mc_dropout=True,weight_regularizer=wr, dropout_regularizer=dr)
x8 = layer_6(x5)
x8 = LeakyReLU(negative_slope=0.01)(x8)

#Fully connected layer and forecast output#
layer_7 = Dense(n_forecast, use_bias=True)
layer_7 = ConcreteDenseDropout(layer_7, is_mc_dropout=True, weight_regularizer=wr, dropout_regularizer=dr,name="output_forecast")
output_forecast = layer_7(x8)

#Welford layer and concatenation layer#
num_series = 24000
welford_forecast_layer = WelfordMeanVarianceLayer(total_series=num_series,n_forecast= n_forecast , name="Welford")
output_forecast_final = welford_forecast_layer([output_forecast, series_id_input])
concatenated_2 = Concatenate(axis=1)([x6_final, output_forecast_final])

#Boltzmann convolutional layer and GAP layer#
x_classification = BoltzmannConv1D(out_channels=32, kernel_sizes=kernel_sizes,dilation_rate=8, wr = wr, dr = dr)(concatenated_2)
x_classification = Activation("relu", name="activation_4")(x_classification)
x_classification = GlobalAveragePooling1D()(x_classification)

#Fully connected layer and classification output#
final_classification_layer = Dense(n_classes, use_bias=False)
final_classification_layer = ConcreteDenseDropout(final_classification_layer, is_mc_dropout=True, weight_regularizer=wr,
                                                  dropout_regularizer=dr,
                                                  name="classification_output")
output_classification = final_classification_layer(x_classification)
output_classification = Activation("softmax")(output_classification)

#compilation and summary of ForeClassNet#
model = Model(inputs=[ts_input, series_id_input],outputs=[output_forecast, output_classification]
              ,name="time_series_forecasting_classification_model")
model.compile(optimizer="adam",loss=["mse", "categorical_crossentropy"],metrics=["mse", "accuracy"])
model.summary()

Model: "time_series_forecasting_classification_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ ts_input            │ (None, 40, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ predefined_boltzma… │ (None, 40, 3)     │        394 │ ts_input[0][0]    │
│ (PredefinedBoltzma… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d    │ (None, 40, 128)   │     17,422 │ ts_input[0][0]    │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_predefi… │ (None, 40, 3)     │          0 │ predefined_boltz… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_learnab… │ (None, 40, 128)   │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40, 131)   │          0 │ activation_prede… │
│ (Concatenate)       │                   │            │ activation_learn… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_1  │ (None, 40, 64)    │  1,081,998 │ concatenate[0][0] │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 40, 64)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ boltzmann_conv1d_2  │ (None, 40, 32)    │    264,430 │ activation_2[0][… │
│ (BoltzmannConv1D)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 40, 32)    │          0 │ boltzmann_conv1d… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 40, 1)     │         33 │ activation_3[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 40)        │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concrete_dense_dro… │ (None, 32)        │      1,313 │ flatten[0][0]     │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 32)        │          0 │ concrete_dense_d… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 40, 1)     │          0 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_forecast     │ (None, 10)        │        331 │ leaky_re_lu[0][0] │
│ (ConcreteDenseDrop… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 40, 1)     │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ series_id           │ (None)            │          0 │ -               

 Total params: 1,878,480 (7.17 MB)

 Trainable params: 1,374,093 (5.24 MB)

 Non-trainable params: 504,387 (1.92 MB)

In [28]:
batch_size = 64
epochs = 100
tic = time.perf_counter()
series_id_train = np.arange(Ns, dtype=np.int32)
model.fit(x=[combined_X_train, series_id_train],  y=[combined_X_forecast_train, combined_y_train], batch_size=batch_size,
    epochs=epochs,validation_split=0.1,shuffle=True)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
time_in_mins = round((toc - tic) / 60)
print("Time in minutes:", time_in_mins)

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 129ms/step - activation_accuracy: 0.5086 - activation_loss: 0.6931 - loss: 2.0647 - output_forecast_loss: 1.4880 - output_forecast_mse: 1.4880 - val_activation_accuracy: 0.5006 - val_activation_loss: 0.6935 - val_loss: 2.0585 - val_output_forecast_loss: 1.4935 - val_output_forecast_mse: 1.4935
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 32s 142ms/step - activation_accuracy: 0.5473 - activation_loss: 0.6749 - loss: 2.0372 - output_forecast_loss: 1.4950 - output_forecast_mse: 1.4950 - val_activation_accuracy: 0.7344 - val_activation_loss: 0.5188 - val_loss: 1.8721 - val_output_forecast_loss: 1.4983 - val_output_forecast_mse: 1.4983
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 33s 146ms/step - activation_accuracy: 0.7650 - activation_loss: 0.4883 - loss: 1.8396 - output_forecast_loss: 1.5003 - output_forecast_mse: 1.5003 - val_activation_accuracy: 0.8150 - val_activation_loss: 0.4202 - val_loss: 1.7571 - val_output_forecast_loss: 1.4976 - val_output_f

In [29]:
epsilon = 0.1  
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
adversarial_test_set_new_new = generate_adversarial_examples(model, X_observed_test,test_series_ids,y_test )

In [30]:
#(b)
n_test_series = X_observed_test.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,X_observed_test,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)
#(a)
n_test_series = adversarial_test_set.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)
#(c)
n_test_series = adversarial_test_set_new.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set_new,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)
#(d)
n_test_series = adversarial_test_set_new_new.shape[0]
test_series_ids = np.arange(n_test_series, dtype=np.int32)
welford_forecast_layer = model.get_layer("Welford") 
welford_forecast_layer.reset_statistics()
(mean_forecast, forecast_uncertainty), (mean_classification, classification_uncertainty) = mc_dropout_predict(
    model,adversarial_test_set_new_new,    test_series_ids,    num_samples=100)
predicted_classes = np.argmax(mean_classification, axis=-1) #obtain the predicted classess 
print("Predicted classes:", predicted_classes)
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predicted_classes) 
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predicted_classes) #assess classification performance
print(classification__report)

MC Dropout passes: 100%|██████████████████████| 100/100 [03:01<00:00,  1.82s/it]


Predicted classes: [0 1 0 ... 1 0 1]
0.9945
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       991
           1       0.99      1.00      0.99      1009

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



MC Dropout passes: 100%|██████████████████████| 100/100 [02:56<00:00,  1.77s/it]


Predicted classes: [0 1 0 ... 1 0 1]
0.994
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       991
           1       0.99      1.00      0.99      1009

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



MC Dropout passes: 100%|██████████████████████| 100/100 [02:53<00:00,  1.74s/it]


Predicted classes: [0 0 0 ... 0 0 0]
0.509
              precision    recall  f1-score   support

           0       0.50      0.97      0.66       991
           1       0.65      0.06      0.11      1009

    accuracy                           0.51      2000
   macro avg       0.57      0.51      0.39      2000
weighted avg       0.57      0.51      0.38      2000



MC Dropout passes: 100%|██████████████████████| 100/100 [02:54<00:00,  1.74s/it]

Predicted classes: [0 0 0 ... 0 0 0]
0.498
              precision    recall  f1-score   support

           0       0.50      0.98      0.66       991
           1       0.56      0.02      0.05      1009

    accuracy                           0.50      2000
   macro avg       0.53      0.50      0.35      2000
weighted avg       0.53      0.50      0.35      2000



In [31]:
clf = 0 
clf = InceptionTimeClassifier(n_epochs=100, batch_size = 64)  
tic = time.perf_counter() 
clf.fit(combined_X_train.transpose(0,2,1), np.argmax(combined_y_train,axis = 1)) 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds") 
#(b)
predictions = clf.predict(X_observed_test.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)
#(a)
predictions = clf.predict(adversarial_test_set.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)
#(c)
predictions = clf.predict(adversarial_test_set_new.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)
#(d)
predictions = clf.predict(adversarial_test_set_new_new.transpose(0,2,1)) 
classification_accuracy = accuracy_score(np.argmax(y_test, axis = 1),predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)  
print(classification__report)

Time taken for fitting: 1355.26 seconds
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Classification Test Accuracy: 0.9125
              precision    recall  f1-score   support

           0       0.87      0.96      0.92       991
           1       0.96      0.86      0.91      1009

    accuracy                           0.91      2000
   macro avg       0.92      0.91      0.91      2000
weighted avg       0.92      0.91      0.91      2000

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Classification Test Accuracy: 0.9125
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       991
           1       0.96      0.86      0.91      1009

    accuracy                           0.91      2000
   macro avg       0.92      0.91      0.91      2000
weighted avg       0.92      0.91      0.91      2000

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Classification Test Accuracy: 0.6065
              precision    recall  f1-score   support

           0       0

In [32]:
MRhydra = 0 
MRhydra = MultiRocketHydraClassifier()  
MRhydra.fit(combined_X_train.transpose(0, 2, 1), np.argmax(combined_y_train,axis = 1))  
#(b)
predictions = MRhydra.predict(X_observed_test.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions = MRhydra.predict(adversarial_test_set.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions = MRhydra.predict(adversarial_test_set_new.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(d)
predictions = MRhydra.predict(adversarial_test_set_new_new.transpose(0, 2, 1))  
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(classification_accuracy)
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

0.7615
              precision    recall  f1-score   support

           0       0.75      0.77      0.76       991
           1       0.77      0.75      0.76      1009

    accuracy                           0.76      2000
   macro avg       0.76      0.76      0.76      2000
weighted avg       0.76      0.76      0.76      2000

0.7505
              precision    recall  f1-score   support

           0       0.75      0.74      0.75       991
           1       0.75      0.76      0.75      1009

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000

0.7155
              precision    recall  f1-score   support

           0       0.70      0.74      0.72       991
           1       0.73      0.69      0.71      1009

    accuracy                           0.72      2000
   macro avg       0.72      0.72      0.72      2000
weighted avg       0.72      0.72      0.72      2000

In [33]:
early = 0 
early = TEASER(classification_points=[40])
tic = time.perf_counter()
early.fit(combined_X_train.transpose(0,2,1), np.argmax(combined_y_train,axis = 1)) 
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
#(b)
predictions, decisions = early.predict(X_observed_test.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions, decisions = early.predict(adversarial_test_set.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions, decisions = early.predict(adversarial_test_set_new.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(d)
predictions, decisions = early.predict(adversarial_test_set_new_new.transpose(0,2,1))  
classification_accuracy = accuracy_score(np.argmax(y_test,axis = 1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 271.51 seconds
Classification Test Accuracy: 0.74
              precision    recall  f1-score   support

           0       0.73      0.75      0.74       991
           1       0.75      0.73      0.74      1009

    accuracy                           0.74      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.74      0.74      0.74      2000

Classification Test Accuracy: 0.7075
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       991
           1       0.72      0.69      0.71      1009

    accuracy                           0.71      2000
   macro avg       0.71      0.71      0.71      2000
weighted avg       0.71      0.71      0.71      2000

Classification Test Accuracy: 0.704
              precision    recall  f1-score   support

           0       0.69      0.72      0.71       991
           1       0.71      0.69      0.70      1009

    accuracy                       

In [34]:
hc2 = 0 
hc2 = HIVECOTEV2(time_limit_in_minutes= time_in_mins)
y_train_labels = np.argmax(combined_y_train, axis=1)
tic = time.perf_counter()
hc2.fit(combined_X_train.transpose(0, 2, 1),y_train_labels)
toc = time.perf_counter()
print(f"Time taken for fitting: {toc - tic:.2f} seconds")
#(b)
predictions = hc2.predict(X_observed_test.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(a)
predictions = hc2.predict(adversarial_test_set.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(c)
predictions = hc2.predict(adversarial_test_set_new.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)
#(d)
predictions = hc2.predict(adversarial_test_set_new_new.transpose(0, 2, 1))
classification_accuracy = accuracy_score(np.argmax(y_test, axis=1), predictions)
print(f"Classification Test Accuracy: {classification_accuracy}")
classification__report = classification_report(np.argmax(y_test, axis=1),predictions)
print(classification__report)

Time taken for fitting: 14677.20 seconds
Classification Test Accuracy: 0.8575
              precision    recall  f1-score   support

           0       0.89      0.81      0.85       991
           1       0.83      0.90      0.86      1009

    accuracy                           0.86      2000
   macro avg       0.86      0.86      0.86      2000
weighted avg       0.86      0.86      0.86      2000

Classification Test Accuracy: 0.825
              precision    recall  f1-score   support

           0       0.85      0.78      0.82       991
           1       0.80      0.87      0.83      1009

    accuracy                           0.82      2000
   macro avg       0.83      0.82      0.82      2000
weighted avg       0.83      0.82      0.82      2000

Classification Test Accuracy: 0.781
              precision    recall  f1-score   support

           0       0.79      0.76      0.78       991
           1       0.78      0.80      0.79      1009

    accuracy                    